In [ ]:
#read dfs# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
quan_train_x=pd.read_csv("../input/garanti-mustafa/quan_train_x.csv")
quan_train_y=pd.read_csv("../input/garanti-mustafa/quan_train_y.csv")
quan_test=pd.read_csv("../input/garanti-mustafa/quan_test.csv")

label_enc_train_x=pd.read_csv("../input/garanti-mustafa/label_enc_train_x.csv")
label_enc_train_y=pd.read_csv("../input/garanti-mustafa/label_enc_train_y.csv")
label_enc_test=pd.read_csv("../input/garanti-mustafa/label_enc_test.csv")

onehot_train_x=pd.read_csv("../input/garanti-mustafa/onehot_train_x.csv")
onehot_train_y=pd.read_csv("../input/garanti-mustafa/onehot_train_y.csv")
onehot_test=pd.read_csv("../input/garanti-mustafa/onehot_test.csv")

onehot_df=pd.read_csv("../input/garanti-mustafa/onehot_df.csv")
quantile_df=pd.read_csv("../input/garanti-mustafa/quantile_df.csv")

In [ ]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error

def xgboostcv(max_depth,
              learning_rate,
              n_estimators,
              gamma,
              min_child_weight,
              max_delta_step,
              subsample,
              reg_lambda,
              colsample_bytree,
              verbosity=0,
              nthread=-1):
    
    model=xgb.XGBRegressor(max_depth=int(round(max_depth)),
                           learning_rate=learning_rate,
                           n_estimators=int(round(n_estimators)),
                           verbosity=verbosity,
                           nthread=nthread,
                           gamma=gamma,
                           min_child_weight=min_child_weight,
                           max_delta_step=int(round(max_delta_step)),
                           subsample=subsample,
                           reg_lambda=reg_lambda,
                           colsample_bytree=colsample_bytree,
                           tree_method='gpu_hist')
    
    result = cross_val_score(model,onehot_train_x,onehot_train_y,scoring="neg_root_mean_squared_error",cv=5).mean()
    
    return result


# Load data set and target values
xgboostBO = BayesianOptimization(f = xgboostcv,
                                 pbounds = {'max_depth': (5, 10),
                                          'learning_rate': (0.01, 0.3),
                                          'n_estimators': (50, 1000),
                                          'gamma': (1., 0.01),
                                          'min_child_weight': (2, 10),
                                          'max_delta_step': (0, 0.1),
                                          'subsample': (0.6, 0.8),
                                          'reg_lambda': (0, 1.),
                                          'colsample_bytree' :(0.5, 0.99)
                                          },
                                random_state=1)

xgboostBO.maximize(init_points=10, n_iter=100)
print('-'*53)

print('Final Results')
for res in xgboostBO.res:
    print('XGBOOST: %f' % res['target'])

print('*'*53)
print('best params ==>', xgboostBO.max)

In [ ]:
param_dict = xgboostBO.max['params'] 
param_dict['tree_method']='gpu_hist'

# we need to round some float values to integer since these parameters works only in integer
param_dict['max_depth']=int(round(param_dict['max_depth']))
param_dict['n_estimators']=int(round(param_dict['n_estimators']))
param_dict['max_delta_step']=int(round(param_dict['max_delta_step']))

In [ ]:
best_xgb_model = xgb.XGBRegressor()
best_xgb_model.set_params(**param_dict)

In [ ]:
best_xgb_model.fit(onehot_train_x,onehot_train_y)


In [ ]:
y_pred = best_xgb_model.predict(onehot_test)


In [ ]:
# prediction results
submission_df=pd.DataFrame(zip(onehot_test["customer_id"],y_pred), columns=["Id", "Predicted"])
submission_df.to_csv('onehot.csv', index=False)